# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

***Participantes (RM - NOME):***<br>
350565 - Emerson Faria de Oliveira<br>
349639 - Caio Lima Uno So<br>


### **Crie um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One).

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

In [1]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

###**Area de desenvolvimento e validações**

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

In [3]:
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


### Drop das colunas não utilizadas e check da distribuição das classes

In [4]:
df.drop(["data_abertura"],axis=1, inplace=True)
df['categoria'].value_counts()

categoria
Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: count, dtype: int64

## Baseline
### Contagem de termos simples com unigrama e sem stop-words.

In [5]:
import nltk
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.25,
      random_state = 42
)

nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')

# vetorização
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.descricao_reclamacao)
text_vect_train = vect.transform(df_train.descricao_reclamacao)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.descricao_reclamacao)
y_test = df_test["categoria"]

# Calculo das metricas
y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Acuracia: 0.7327
Precisao: 0.7336
Recall: 0.7327
F1 Score: 0.7329


### [Experimento 1] - Contagem de termos simples com unigrama + brigrama e sem stop-words.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
import nltk

nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')

# vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df_train.descricao_reclamacao)
text_vect_train = vect.transform(df_train.descricao_reclamacao)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.descricao_reclamacao)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Acuracia: 0.7392
Precisao: 0.7394
Recall: 0.7392
F1 Score: 0.7389


### [Experimento 2] - TF-IDF com unigrama e com stop-words.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

# vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df_train.descricao_reclamacao)
text_vect_train = vect.transform(df_train.descricao_reclamacao)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.descricao_reclamacao)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

Acuracia: 0.7578
Precisao: 0.7578
Recall: 0.7578
F1 Score: 0.7577


### [Experimento 3] - TF-IDF com bigrama e sem stop-words.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
import nltk

nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')

# vetorização
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops)
vect.fit(df_train.descricao_reclamacao)
text_vect_train = vect.transform(df_train.descricao_reclamacao)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.descricao_reclamacao)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Acuracia: 0.7589
Precisao: 0.7605
Recall: 0.7589
F1 Score: 0.7593


### [Experimento 4] - TF-IDF com unigrama e sem stop-words em textos lematizados.

In [9]:
# teste das funções de lematização
import spacy
nlp = spacy.load('pt_core_news_sm')

# função de lematização completa do documento
def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

# validação das funções
print(lemmatizer_text('correndo 1, 2, 3'))

correr 1 , 2 , 3


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
import nltk

nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')

# aplica a lematização no dataframe de treino criando um nova coluna
df_train['text_lemma'] = df_train.descricao_reclamacao.apply(lemmatizer_text)

# vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vect.fit(df_train.text_lemma)
text_vect_train = vect.transform(df_train.text_lemma)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
# aplica a lematização no dataframe de treino criando um nova coluna
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(lemmatizer_text)
x_test = vect.transform(df_test.text_lemma)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Acuracia: 0.7747
Precisao: 0.7746
Recall: 0.7747
F1 Score: 0.7746


### [Experimento 5] - TF-IDF com bigrama e sem stop-words (SpaCy e NLTK) em textos lematizados e normalizados. **(Campeão)**

In [11]:
import re 
import nltk
import spacy
import string
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

nlp = spacy.load('pt_core_news_sm')

# stopwords do SpaCy e NLTK combinadas
stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords =  stopwords + ['xxxx', 'xxx', 'xx' ,'chase']

def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    text = " ".join([t for t in text])
    return text

# aplica a lematização no dataframe de treino criando um nova coluna
df_train['text_lemma'] = df_train.descricao_reclamacao.apply(norm_tokenize)
df_train['text_lemma'] = df_train.descricao_reclamacao.apply(lemmatizer_verbs)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# vetorização
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops)
vect.fit(df_train.text_lemma)
text_vect_train = vect.transform(df_train.text_lemma)

x_train = text_vect_train
y_train = df_train["categoria"]

# treinamento do modelo
# model = DecisionTreeClassifier(random_state=42)
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
# aplica a lematização no dataframe de treino criando um nova coluna
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(norm_tokenize)
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(lemmatizer_verbs)

x_test = vect.transform(df_test.text_lemma)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

Acuracia: 0.9011
Precisao: 0.9016
Recall: 0.9011
F1 Score: 0.9007


### [Experimento 6] - Classificador com word2vec (Normalização/Tokenização)

In [ ]:
# Download do arquivo no repositório do professor
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip'

# Descompactação do arquivo
!unzip 'skip_s300.zip' # subistitua com nome do arquivo
!ls -la

In [13]:
# Load do modelo pelo Gensim
from gensim.models import KeyedVectors

model_skip = KeyedVectors.load_word2vec_format('skip_s300.txt')
model_skip

In [14]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Normalização de texto
import nltk
import string
import numpy as np
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# lista de stopwords do NLTK
stopwords = nltk.corpus.stopwords.words('portuguese')

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    #return " ".join([t for t in text])
    return text

# Função para obter embeddings médios para cada texto
vectorizer = model_skip

def average_vector(words):
  vectors = [vectorizer[word] for word in words if word in vectorizer]
  if vectors:
    return sum(vectors) / len(vectors)
  else:
    vector_size = vectorizer.vector_size
    wv_res = np.zeros(vector_size)
    return wv_res

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Tokenizando e normalizando o texto
df_train['tokens'] = df_train.descricao_reclamacao.apply(norm_tokenize) 
# Converter textos para embeddings com Word2Vec
df_train["vetor"] = df_train["tokens"].apply(average_vector)

# variáveis
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Classificador
model = LogisticRegression(random_state=42)
# model = RandomForestClassifier(random_state=42)
model.fit(list(x_train), y_train)

# Avaliação do modelo
# Tokenizando e normalizando o texto
df_test['tokens'] = df_test.descricao_reclamacao.apply(norm_tokenize) 
# Converter textos para embeddings com Word2Vec
df_test["vetor"] = df_test["tokens"].apply(average_vector)

x_test = df_test["vetor"]
y_test = df_test["categoria"]

y_pred = model.predict(list(x_test))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print()
print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))

c:\Users\nosre\Desktop\GIT\natural-language-procesing\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.87      0.85      0.86      1290
            Hipotecas / Empréstimos       0.84      0.87      0.85       922
                             Outros       0.85      0.71      0.78       549
       Roubo / Relatório de disputa       0.79      0.82      0.80      1204
         Serviços de conta bancária       0.86      0.90      0.88      1303

                           accuracy                           0.84      5268
                          macro avg       0.84      0.83      0.83      5268
                       weighted avg       0.84      0.84      0.84      5268

0.8432042520880789

Acuracia: 0.8432
Precisao: 0.8438
Recall: 0.8432
F1 Score: 0.8426


### [Experimento 7] - LDA com unigrama e sem stop-words (SpaCy e NLTK) em textos lematizados.

In [17]:
import nltk
import spacy
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')

# stopwords do SpaCy e NLTK combinadas
stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords =  stopwords + ['xxxx']

def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

df_train['text_lemma'] = df_train.descricao_reclamacao.apply(lemmatizer_verbs)
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(lemmatizer_verbs)

# vetorização
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.text_lemma)

text_vect_train = vect.transform(df_train.text_lemma)

# modelo
LDA = LatentDirichletAllocation(n_components=5,random_state=42)
LDA.fit(text_vect_train)

# top palavras dos tópicos
terms = vect.get_feature_names_out()

for index, topic in enumerate(LDA.components_):
    terms_comp = zip(terms, topic)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print("THE TOP 15 WORDS FOR TOPIC # "+str(index)+": ")
    #print(sorted_terms)
    print([t[0] for t in sorted_terms])
    print('\n')

text_vect_test = vect.transform(df_test.text_lemma)
results = LDA.transform(text_vect_test)

df_test['topico'] = results.argmax(axis=1)

df_test['categ_cod'] = df_test.categoria.map(
    {'Serviços de conta bancária':1 , 'Cartão de crédito / Cartão pré-pago':0 , 'Roubo / Relatório de disputa':2 , 'Hipotecas / Empréstimos': 3, "Outros":4 }
)

print(accuracy_score(df_test.categ_cod, df_test.topico))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


THE TOP 15 WORDS FOR TOPIC # 0: 
['00', 'pagamento', 'pagar', 'crédito', 'saldo', 'cartão', 'taxa', 'chase', 'juros', 'taxas', 'pagamentos', 'cobrar', 'conta', 'ter', 'valor']


THE TOP 15 WORDS FOR TOPIC # 1: 
['conta', 'xxxx', 'chase', 'dizer', 'ter', 'fazer', 'poder', 'dinheiro', 'xx', 'banco', 'cartão', '00', 'cheque', 'então', 'fundos']


THE TOP 15 WORDS FOR TOPIC # 2: 
['chase', 'conta', 'crédito', 'xx', 'taxa', 'informações', 'lei', 'fornecer', 'consumidor', 'contrato', 'dever', 'ter', 'qualquer', 'taxas', 'dias']


THE TOP 15 WORDS FOR TOPIC # 3: 
['xxxx', 'chase', 'crédito', 'ter', 'empréstimo', 'hipoteca', 'casa', 'fazer', 'relatório', 'poder', 'anos', 'pagamentos', 'dizer', 'pagar', 'nunca']


THE TOP 15 WORDS FOR TOPIC # 4: 
['xxxx', 'xx', 'chase', '00', 'cartão', 'crédito', 'dizer', 'ter', 'enviar', 'conta', 'receber', 'recebir', 'contato', 'fazer', 'poder']


0.38686408504176156


### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

In [2]:
import nltk
import spacy
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# CARREGANDO O DATA FRAME
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df.drop(["data_abertura"],axis=1, inplace=True)

In [4]:
# Divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.25,
      random_state = 42
)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

# stopwords do SpaCy e NLTK combinadas
stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords =  stopwords + ['xxxx', 'xxx', 'xx' ,'chase']

def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    text = " ".join([t for t in text])
    return text

In [ ]:

# Aplica a lematização no dataframe de treino criando um nova coluna
df_train['text_lemma'] = df_train.descricao_reclamacao.apply(norm_tokenize)
df_train['text_lemma'] = df_train.descricao_reclamacao.apply(lemmatizer_verbs)

# Vetorização
vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stops)
vect.fit(df_train.text_lemma)
text_vect_train = vect.transform(df_train.text_lemma)

x_train = text_vect_train
y_train = df_train["categoria"]

# Treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# Teste do modelo
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(norm_tokenize)
df_test['text_lemma'] = df_test.descricao_reclamacao.apply(lemmatizer_verbs)

x_test = vect.transform(df_test.text_lemma)
y_test = df_test["categoria"]

y_pred = model.predict(x_test)

# Metricas
print(" ----------- Metricas do modelo campeao ----------- ")
print("Acuracia: %.4f" %(accuracy_score(y_test, y_pred)))
print("Precisao: %.4f" % (precision_score(y_test, y_pred, average='weighted')))
print("Recall: %.4f" % (recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: %.4f" % (f1_score(y_test, y_pred, average='weighted')))